<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2024TeamA/blob/main/Abfragen_DWH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notwendige Schritte, um das DWH zu öffnen. Pfad muss ggf. angepasst oder auskommentiert werden. Dieser Abschnitt muss einmal ausgeführt werden, um sich mit dem DWH zu verbinden. Danach können die Abfragen
einzeln ausgeführt werden.



In [ ]:
# !pip3 install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
import pandas as pd
import numpy as np
import sqlite3
# import pandas_profiling
# from datetime import datetime
# import re
import seaborn as sns
import matplotlib.pyplot as plt
# from pandas_profiling import ProfileReport
# will make plot outputs appear and stored within the notebook.
%matplotlib inline
from sqlite3 import Error

from google.colab import drive
drive.mount('/content/drive')

# Path
material_path = "/content/drive/MyDrive/M3_DM_GrpA"

# path of datawarehouse
DB_DWH_PATH = material_path + f"/DWH_GrpA.db"

dwh_db = sqlite3.connect(DB_DWH_PATH)
dwh_cursor = dwh_db.cursor()

Unabhängige Abfragen zur Nutzung

In [ ]:
# check list of tables
dwh_cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(dwh_cursor.fetchall())

In [ ]:
# check columns in a table (hier: county_info)
dwh_cursor.execute('PRAGMA table_info(' + "county_info" + ');')
dwh_cursor.fetchall()

In [ ]:
# Schreibe ein SQL Statement und lass die ersten 10 Zeilen ausgeben
# Bsp 1
dwh_cursor.execute("SELECT Id, BIRTHDATE, FIRST, LAST, MARITAL, GENDER from patients_info")
rows = dwh_cursor.fetchall()
for row in rows[:10]:
  print(row)

In [ ]:
# Bsp 2
dwh_cursor.execute("SELECT COUNT(DISTINCT PATIENT),CODE, DESCRIPTION STRING, BASE_COST from procedures_info WHERE DESCRIPTION LIKE '%OXYGEN%' GROUP BY CODE")
rows = dwh_cursor.fetchall()
for row in rows[:10]:
  print(row)

In [ ]:
dwh_cursor.execute("SELECT p.Id, COUNT(e.Id) AS Anzahl, e.ENCOUNTERCLASS FROM encounters_info e JOIN patients_info p ON e.PATIENT = p.Id WHERE e.REASONDESCRIPTION = 'COVID-19' GROUP BY p.Id")
rows = dwh_cursor.fetchall()
print(dwh_cursor.description)
for row in rows[:10]:
  print(row)

Abfrage: County, Density, Anzahl Covid

In [ ]:
dwh_cursor.execute("SELECT patients_info.COUNTY, county_info.DENSITY_per_sqm, COUNT(patients_info.Id), conditions_info.DESCRIPTION FROM conditions_info JOIN patients_info, county_info ON conditions_info.PATIENT = patients_info.Id AND patients_info.COUNTY = county_info.COUNTY WHERE conditions_info.DESCRIPTION = 'COVID-19' GROUP BY patients_info.COUNTY")
rows = dwh_cursor.fetchall()
for row in rows:
  print(row)

In [ ]:
dwh_cursor.execute("SELECT e.ENCOUNTERCLASS, e.START, p.DEATHDATE FROM encounters_info e JOIN patients_info p ON e.PATIENT = p.Id WHERE e.REASONDESCRIPTION = 'COVID-19' ORDER BY DEATHDATE DESC")
rows = dwh_cursor.fetchall()
for row in rows[:10]:
  print(row)

SQL Abfrage mit Pandas

In [ ]:
df_patients = pd.read_sql_query("SELECT * FROM PATIENTS_INFO", dwh_db)
df_patients.head()

In [ ]:
df_conditions = pd.read_sql_query("SELECT * FROM conditions_INFO", dwh_db)
df_conditions.head()

In [ ]:
df_medications = pd.read_sql_query("SELECT * FROM medications_INFO WHERE REASONDESCRIPTION LIKE '%COV%'", dwh_db)
df_medications.head()

In [ ]:
df_devices = pd.read_sql_query("SELECT DESCRIPTION FROM devices_INFO GROUP BY DESCRIPTION ", dwh_db)
print(df_devices)